In [ ]:
# Im adding the heart rate into the features extracted using a heuristic function based on these rules 

# Pitch (F0) — higher pitch means higher heart rate
# Energy — louder speech means higher heart rate
# Speaking rate — faster speech means higher heart rate

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("speech_features.csv")

In [ ]:
import numpy as np

def simulate_heart_rate(row):
    resting_hr = 65 + 10 * np.random.rand()  # variable resting HR between 65-75
    
    pitch_norm = min(max(row['pitch_mean'] / 250, 0), 1)
    energy_norm = min(max(row['energy'] / 10, 0), 1)
    rate_norm = min(max(row['speaking_rate'] / 5, 0), 1)
    
    pitch_effect = 20 / (1 + np.exp(-12 * (pitch_norm - 0.5)))
    energy_effect = 15 * energy_norm ** 2
    rate_effect = 10 * np.sqrt(rate_norm)
    
    interaction = 10 * pitch_norm * rate_norm
    
    hr = resting_hr + pitch_effect + energy_effect + rate_effect + interaction
    
    noise = np.random.normal(0, 3)
    hr += noise
    
    return np.clip(hr, 50, 160)


In [7]:
df["simulated_hr"] = df.apply(simulate_heart_rate, axis=1)

df.to_csv("speech_features_with_hr.csv", index=False)

print("done.")

done.
